In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import seaborn as sns

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import KFold, StratifiedKFold

warnings.filterwarnings('ignore')



In [2]:
""""
# coffee data
url="https://github.com/jldbc/coffee-quality-database/raw/master/data/robusta_data_cleaned.csv"
coffee_features=pd.read_csv(url)

# coffe score

url="https://raw.githubusercontent.com/jldbc/coffee-quality-database/master/data/robusta_ratings_raw.csv"
coffee_quality=pd.read_csv(url)
coffee_quality.head()

Y = coffee_quality["quality_score"]
coffee_features.info()
#for this exercise we will only deal with numeric variables

X = coffee_features.select_dtypes(['number'])
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

#dropping Quakers column and unnamed
#changing one of the altitude to log and droping the original
X_train["altitude_mean_log"] = np.log(X_train["altitude_mean_meters"])
X_train.drop(['altitude_mean_meters'], axis=1, inplace=True)
X_train.drop(['Quakers'], axis=1, inplace=True)
X_train.drop(['Unnamed: 0'], axis=1, inplace=True)

X_train.info()

altitude_low_meters_mean = X_train["altitude_low_meters"].mean()
altitude_high_meters_mean = X_train["altitude_high_meters"].mean()
altitude_mean_log_mean = X_train["altitude_mean_log"].mean()

# fillna with mean.. 
X_train["altitude_low_meters"] = X_train["altitude_low_meters"].fillna(altitude_low_meters_mean)
X_train["altitude_high_meters"] = X_train["altitude_high_meters"].fillna(altitude_high_meters_mean)
X_train["altitude_mean_log"] = X_train["altitude_mean_log"].fillna(altitude_mean_log_mean)

print(f"altitude low meters mean is {altitude_low_meters_mean}")
print(f"altitude_high_meters_mean is {altitude_high_meters_mean}")
print(f"altitude_mean_log_mean is {altitude_mean_log_mean}")

## in order to exemplify how the predict will work.. we will save the y_train
X_test.to_csv("data/X_test.csv")
y_test.to_csv("data/y_test.csv")

#training the model
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)

from sklearn.metrics import mean_squared_error
y_train_pred = reg.predict(X_train)
mse = mean_squared_error(y_train, y_train_pred)
print(mse)

#dropping Quakers column and unnamed
#changing one of the altitude to log and droping the original
X_test["altitude_mean_log"] = np.log(X_test["altitude_mean_meters"])
X_test.drop(['altitude_mean_meters'], axis=1, inplace=True)
X_test.drop(['Quakers'], axis=1, inplace=True)
X_test.drop(['Unnamed: 0'], axis=1, inplace=True)
# fillna with mean.. 
X_test["altitude_low_meters"] = X_test["altitude_low_meters"].fillna(altitude_low_meters_mean)
X_test["altitude_high_meters"] = X_test["altitude_high_meters"].fillna(altitude_high_meters_mean)
X_test["altitude_mean_log"] = X_test["altitude_mean_log"].fillna(altitude_mean_log_mean)

y_test_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_test_pred)
print(mse)

"""

'"\n# coffee data\nurl="https://github.com/jldbc/coffee-quality-database/raw/master/data/robusta_data_cleaned.csv"\ncoffee_features=pd.read_csv(url)\n\n# coffe score\n\nurl="https://raw.githubusercontent.com/jldbc/coffee-quality-database/master/data/robusta_ratings_raw.csv"\ncoffee_quality=pd.read_csv(url)\ncoffee_quality.head()\n\nY = coffee_quality["quality_score"]\ncoffee_features.info()\n#for this exercise we will only deal with numeric variables\n\nX = coffee_features.select_dtypes([\'number\'])\nfrom sklearn.model_selection import train_test_split\n\nX_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)\n\n#dropping Quakers column and unnamed\n#changing one of the altitude to log and droping the original\nX_train["altitude_mean_log"] = np.log(X_train["altitude_mean_meters"])\nX_train.drop([\'altitude_mean_meters\'], axis=1, inplace=True)\nX_train.drop([\'Quakers\'], axis=1, inplace=True)\nX_train.drop([\'Unnamed: 0\'], axis=1, inplace=True)\n\

## Data Exploration

In [3]:
df = pd.read_csv('./data/Train.csv')
pd.set_option('display.max_rows', None)
df.head(5)

,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,...,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.000000,60.200001,...,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.600000,48.799999,...,59.624912,-67.693509,28.614804,1789.960449,3384.226562,0.015104,75.630043,55.657486,-53.868134,19.293652
2,010Q650 X 2020-01-04,2020-01-04,010Q650,24.0,8.0,56.0,1181.96,96,16.400000,33.400002,...,49.839714,-78.342701,34.296977,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,010Q650 X 2020-01-05,2020-01-05,010Q650,49.0,10.0,55.0,1113.67,96,6.911948,21.300001,...,29.181258,-73.896588,30.545446,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,010Q650 X 2020-01-06,2020-01-06,010Q650,21.0,9.0,52.0,1164.82,95,13.900001,44.700001,...,0.797294,-68.612480,26.899694,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(30557, 82)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30557 entries, 0 to 30556
Data columns (total 82 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Place_ID X Date                                      30557 non-null  object 
 1   Date                                                 30557 non-null  object 
 2   Place_ID                                             30557 non-null  object 
 3   target                                               30557 non-null  float64
 4   target_min                                           30557 non-null  float64
 5   target_max                                           30557 non-null  float64
 6   target_variance                                      30557 non-null  float64
 7   target_count                                         30557 non-null  int64  
 8   precipitable_water_entire_atmosphere                 30557 non-nul

In [6]:
df.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
target,30557.0,61.15,46.86,1.00,25.00,50.00,80.00,815.00
target_min,30557.0,29.03,33.12,1.00,5.00,15.00,44.00,438.00
target_max,30557.0,117.99,100.42,1.00,60.00,91.00,155.00,999.00
target_variance,30557.0,7983.76,48630.90,0.00,1064.92,2395.35,5882.55,1841490.00
target_count,30557.0,125.83,146.58,2.00,44.00,72.00,150.00,1552.00
precipitable_water_entire_atmosphere,30557.0,15.30,10.69,0.42,7.67,12.20,19.90,72.60
relative_humidity_2m_above_ground,30557.0,70.55,18.81,5.13,58.60,74.10,85.45,100.00
specific_humidity_2m_above_ground,30557.0,0.01,0.00,0.00,0.00,0.00,0.01,0.02
temperature_2m_above_ground,30557.0,9.32,9.34,-34.65,3.12,8.48,16.20,37.44
u_component_of_wind_10m_above_ground,30557.0,0.42,2.71,-15.56,-1.10,0.22,1.77,17.96


In [7]:
# There are 340 locations
df.Place_ID.nunique()

340

In [8]:
df.Date.groupby(df.Place_ID).nunique().sort_values(ascending=False)
# Number of dates measured for each Place ID

Place_ID
010Q650    94
JSXAVKO    94
I5RGE5G    94
I6718VY    94
I6VIR8R    94
IJKTQ0G    94
IM6VR3X    94
IOLPRPO    94
IST2SPS    94
IXVOHYQ    94
J39EIKN    94
J3HFUPO    94
J4UZBX3    94
J8IR9R2    94
JABMGKI    94
JID7N54    94
WQP8SI3    94
HRUE894    94
KB7LD9K    94
KXX7F64    94
KYNZWCA    94
KZJQU9F    94
L17O89H    94
L686B2W    94
LF5EAJD    94
WP7PTYQ    94
LXRFONN    94
M17FYGT    94
M53IXJM    94
WOIRN9J    94
MUJVVNA    94
N5RI4GT    94
HSXRMLA    94
HR0OKCA    94
DCBEVTO    94
F9780KK    94
DPQKOA5    94
DRFD4D1    94
DSG5LHH    94
DXU9CSJ    94
E05B6AD    94
E3IZI9K    94
E3VXRRJ    94
E7N9S0X    94
E9KQB8A    94
ECCAPWQ    94
EE8C6RP    94
EVB4HVG    94
VHZ9CYS    94
F51ZEA7    94
FAYY7HV    94
HN3YMTM    94
FFK3RNL    94
FH6B9N5    94
FLN66MS    94
FOQIF0X    94
FUTHSQ2    94
FW7AVMW    94
G4CKKFV    94
GGVKSLX    94
GNHAWDW    94
GP8V3PO    94
GSIMFXX    94
GU2Z4N6    94
H6JSAYZ    94
H7QOMTD    94
NCIFTFE    94
NHB348L    94
NJ1G8AW    94
TSWW4S1    94
S9GAHFC    

In [9]:
# Checking for 0 values
df_pp = df.copy()
missing = pd.DataFrame((df_pp==0).sum(), columns=["Zero_Amount"])
missing.head()
missing['Percentage'] = round((missing['Zero_Amount']/df.shape[0])*100, 2)
missing[missing['Zero_Amount'] != 0].sort_values(by = 'Percentage', ascending=False)

,Zero_Amount,Percentage
L3_CH4_solar_zenith_angle,2915,9.54
L3_CH4_sensor_azimuth_angle,2915,9.54
L3_CH4_solar_azimuth_angle,2915,9.54
L3_CH4_aerosol_optical_depth,2915,9.54
L3_CH4_aerosol_height,2915,9.54
L3_CH4_CH4_column_volume_mixing_ratio_dry_air,2915,9.54
L3_CH4_sensor_zenith_angle,2915,9.54
L3_NO2_cloud_fraction,2897,9.48
L3_NO2_sensor_zenith_angle,2611,8.54
L3_NO2_solar_azimuth_angle,2611,8.54


# Data cleaninging and feature engineering

In [12]:
# Changing the Date column to a datetime variable
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

# We will create a new columns with the frequency of the places
df['PlaceID_freq'] = df['Place_ID'].map(df['Place_ID'].value_counts())

# No feature columns
no_features = ['Place_ID X Date', 'Date', 'Place_ID', 'target', 'target_min'
               'target_max', 'target_variance', 'target_count', 'PlaceID_freq']

features = [f for f in df.columns if f not in no_features] # This columns help to identify
                                                            # the data, not measurements

## Splitting data for testing 

## Trainining the model